In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [ ]:
spark = SparkSession.builder.appName('Laboratorio COVID-19 Twitter').getOrCreate()
spark

In [90]:
esp_regular_exclutions = ['', 'https://t', 'contra', 'este', 'sobre', 'durante', 'más', 'está', 'ha', 'el', 'la', 'los', 'las', 'un', 'no', 'una', 'unos', 'unas', 'al', 'del', 'lo', 'le', 'y', 'e', 'o', 'u', 'de', 'a', 'en', 'que', 'es',
                          'por', 'para', 'con', 'se', 'su', 'les', 'me', 'q', 'te', 'pero', 'mi', 'ya', 'cuando', 'como', 'estoy', 'voy', 'porque', 'he', 'son', 'solo', 'tengo', 'muy', 'si', 'fue', 'síntomas', 'todos', 'mascarilla', 'personas', 'salud', '#covid19',
                          'virus', 'pacientes', 'casos', 'síntomas,', 'tienen', 'después', 'nueva','tienes', 'enfermedad', 'usar', 'días', 'sintomas', 'coronavirus', 'tiene', 'esta', 'covid,', 'caso', 'lucha', 'prueba', 'ahora', 'puede', 'clave', 'estos', 'todo', 
                          'desde', 'llamada', 'están', 'gente', 'niños','positivo', 'hasta', 'entre', 'hacer', 'hace', 'presentan', 'test', 'tener', 'pandemia', 'muchos', 'conoce', 'menos', 'años', 'noviembre', 'también', 'cómo', 'tuve', 'medidas', 'cada', '\nhttps://t',
                          'corona','capital','mejor','antes','bien','@minsa_peru','pandemia','donde','nada','esto','mucho','verdad','pues','siempre','millones','nunca','igual','mismo','país','algo','mismo','estaba','estés','atención''#covid19,','tres','meses','presentas',
                          'algunos','principales','dosis','atención','pruebas','#covid19,','fiebre,','covid?','veces','problemas','casa','negativo','presentar','tenido','covid.','cura','tratamiento','agua','decir','cáncer','corona','porque','cuando',
                          'síntoma','#COVID19','COVID-19','Conoce','ningún','fueron','población','Síntomas','Covid-19']
esp_key_exclutions = ['covid', 'covid-19', 'vacuna', 'vacunas', 'covid-19,','variante','síntomas.',' evitar','mismos','contagio','semana','tomar','@claudiashein','ivermectina.','medicamentos','cuenta','médicos','covid-19',"#felizcumpleañospdtemaduro"]
total_exclutions = esp_regular_exclutions + esp_key_exclutions

Pregunta #1: Analisis de sentimientos sobre el tema vacunas

In [ ]:
df_sentimientos = spark.read.option('header','true').csv('./data/sentimientos_vacuna_2.csv',inferSchema=True)
count_sentimientos = df_sentimientos.select("sentimiento").groupBy("sentimiento").count().show()

Pregunta #2: Sintomas mas consultados

In [ ]:
df_covid_symptoms = spark.read.option('header','true').csv('./data/symptom_tweets.csv',inferSchema=True)
df_covid_symptoms.select('tweet').where(df_covid_symptoms.entidad == 'COVID-19').show()

In [ ]:
words_symptoms_count = df_covid_symptoms.withColumn('word', f.explode(f.split(f.col('tweet'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)
lower_words_count = words_symptoms_count.select([f.lower(words_symptoms_count.word).alias('word'),'count'])
larger_words_df =  words_symptoms_count.select(['word','count']).where(f.length(words_symptoms_count.word) > 5)
larger_words_df.filter(larger_words_df.word.isin(total_exclutions) == False).show(20, False)

Pregunta #3: Temas más consultados sobre el covid en twitter

In [ ]:
df_covid_tweets = spark.read.option('header','true').csv('./data/covid_tweets.csv',inferSchema=True)
df_covid_tweets = df_covid_tweets.select(f.lower(df_covid_tweets.tweet))
df_covid_tweets.show()

In [ ]:

words_count = df_covid_tweets.withColumn('word', f.explode(f.split(f.col('tweet'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)
words_count = words_count.select([f.lower(words_count.word).alias('word'),'count'])
words_count.show()

In [ ]:
larger_words_df =  words_count.select(['word','count']).where(f.length(words_count.word) > 5)
larger_words_df.filter(larger_words_df.word.isin(total_exclutions) == False).show(20, False)

Pregunta #4: Analisis de sentimiento sobre Azitromicina, dioxido de cloro y invermectina

In [ ]:
df_alternative_medicines = spark.read.option('header','true').csv('./data/4medicina_alternativa.csv',inferSchema=True)
df_alternative_medicines.show()

In [134]:
words_count = df_alternative_medicines.withColumn('medicina_alt', f.explode(f.split(f.col('tweet'), ' ')))\
    .groupBy(f.lower('medicina_alt').alias('medicina_alt'))\
    .count()
alt_filter_df = words_count.filter(words_count.medicina_alt.isin(['azitromicina','dioxido','ivermectina','cloroquina']))
alt_filter_df.show(20, False)

+------------+-----+
|medicina_alt|count|
+------------+-----+
|dioxido     |8    |
|azitromicina|206  |
|cloroquina  |678  |
|ivermectina |3778 |
+------------+-----+

